# Exercise 03 - Gradient descent, linear regression, and interpretability

After completing this notebook, you should be able to:
* implement gradient descent
* have an understanding how linear regression works
* how weights of a linear regression model can be interpreted

## Gradient Descent

We will use the 'digits' dataset consisting of 8x8 images of handwritten digits.

In [36]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets

# generate random dataset
X, y = datasets.make_regression(n_features=1, noise=50, random_state=42)
y = y + 500

# plot data
fig, ax = plt.subplots(1,1)
ax.scatter(X, y)

# TODO: notice the intercept!

In [ ]:
# TODO: complete the linear regression function
def linear_regression(X, w=None, b=0):
    
    if w is None:
        w = np.zeros(X.shape[1])
    
    # TODO: fill in formula for linear regression in matrix notation
    y_pred = ... # TODO
    
    return y_pred

In [2]:
# calculating `y_pred` with weights `[0]` and bias `0`

w = np.array([0])
b = 0

y_pred = linear_regression(X, w=w, b=b)

In [41]:
fig, ax = plt.subplots(1,1)
ax.scatter(X, y, label="data")
ax.scatter(X, y_pred, color="red", label="prediction")

In [42]:
# TODO: implement a function to calculate the gradient for linear regression  (hint: use the matrix multiplication variant)
def linear_regression_derivative(X, y, w, b):
    gradient_w = ... # TODO
    gradient_b = ... # TODO
    return gradient_w, gradient_b

In [ ]:
# TODO: fill in the weight update formula and play with the learning rate, what do you observe?

# starting configuration
w = np.array([0]); b = 0

# learning rate
learning_rate = 0.00001

# number of epochs
n_epochs = 1000

# training loop
for i_epoch in range(n_epochs):
    
    # calculate gradients
    gradient_w, gradient_b = linear_regression_derivative(X, y, w, b)
    
    # update gradients
    w = ... # TODO
    b = ... # TODO

    # plot current predictions    
    if i_epoch % 100 == 0:
        fig, ax = plt.subplots(1,1)
        ax.scatter(X, y)
        y_pred = linear_regression(X, w=w, b=b)
        ax.scatter(X, y_pred, c="green")

# plot final predictions
fig, ax = plt.subplots(1,1)
ax.scatter(X, y)
y_pred = linear_regression(X, w=w, b=b)
ax.scatter(X, y_pred, c="green")

In [43]:
# TODO: Bonus: plot the loss curve!

## Interpreting linear regression

We are working with the [titanic dataset](https://www.kaggle.com/competitions/titanic).

The following code plots feature importances for a linear model predicting passenger survival.
1) Try difference data splits. What do you observe for the feature importances?
1) Try adding and removing individual features. What do you observe?
1) Try scaling individual features and see how it affects the features. What do you observe?
1) Are the features that are considered important according to a model the only possibly important features to make predictions?
1) Sample feature importances using bootstrapping and plot them as boxplots. Which features are significant?

In [44]:
# imports
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, cross_validate, StratifiedKFold, RepeatedStratifiedKFold

from sklearn.pipeline import make_pipeline

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC

import matplotlib.pyplot as plt
import seaborn as sns

In [46]:
# load data
data_titanic = pd.read_csv("titanic.csv", index_col="PassengerId")

In [47]:
def extract_features(data):
    """Extract features from existing variables"""
    
    data_extract = data.copy()
    
    # name
    name_only = data_extract["Name"].str.replace(r"\(.*\)", "", regex=True)
    first_name = name_only.str.split(", ", expand=True).iloc[:,1]
    title = first_name.str.split(".", expand=True).iloc[:,0]
    data_extract["Title"] = title
        
    # ticket
    
    # ...
    
    return data_extract
    
data_extract = extract_features(data_titanic)

In [48]:
def preprocess(data):
    """Convert features into numeric variables readable by our models.""" 
    
    data_preprocessed = data.copy()
    
    # Sex
    
    data_preprocessed = pd.get_dummies(data_preprocessed, columns=["Sex"], drop_first=True)
    
    # Cabin
    
    data_preprocessed.drop(columns="Cabin", inplace=True)
    
    # Embarked
    
    data_preprocessed = pd.get_dummies(data_preprocessed, columns=["Embarked"], dummy_na=True)
    
    # Title
    title = data_preprocessed["Title"]
    title_counts = title.value_counts()
    higher_titles = title_counts[title_counts < 50]
    title_groups = ["higher" if t in higher_titles else t for t in title]
    data_preprocessed["Title"] = title_groups
    data_preprocessed = pd.get_dummies(data_preprocessed, columns=["Title"])
    
    # drop the rest
    data_preprocessed.drop(columns=["Name", "Ticket"], inplace=True)
    
    return data_preprocessed

data_preprocessed = preprocess(data_extract)

In [49]:

# before inspecting the data, selecting and building models, etc. 
# FIRST split data into train and test data (we set the test data size to 30%)

X = data_preprocessed.drop(columns="Survived")
y = data_preprocessed["Survived"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)

In [50]:
imputer = SimpleImputer(strategy="mean")
scaler = StandardScaler()
preprocessing_pipeline = make_pipeline(scaler, imputer)


X_train_example = X_train.copy()

# drop features here
# ...

X_train_processed = preprocessing_pipeline.fit_transform(X_train_example)

# change `X_train_processed` for example here
# ...


# set model
model = LogisticRegression()

# fit for coefficient extraction
model.fit(X_train_processed, y_train)
feature_weights = model.coef_.flatten()

''

In [52]:
# plot feature importances
fig, ax = plt.subplots(1, 1, figsize=(6 * 2, 6), sharey=True)
sns.barplot(x=X_train_example.columns, y=feature_weights, ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), rotation=270)